In [1]:
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
use_gpu = torch.cuda.is_available()
print(device)

cuda:0


In [2]:
a = torch.LongTensor([[1,2,3,4,5]]).to(device)
print(a[0:3].device)
print(a[:,[1,3,4]].device)
print(a.T.device)

cuda:0
cuda:0
cuda:0


# Initialize for CPU & GPU

In [3]:
if device == 'cpu':
    !pip uninstall torch-scatter torch-sparse torch-geometric torch-cluster  --y
    !pip install torch-scatter -f https://data.pyg.org/whl/torch-{torch.__version__}.html
    !pip install torch-sparse -f https://data.pyg.org/whl/torch-{torch.__version__}.html
    !pip install torch-cluster -f https://data.pyg.org/whl/torch-{torch.__version__}.html
    !pip install git+https://github.com/pyg-team/pytorch_geometric.git
#     !pip install torch_geometric_temporal
else:
    !pip install torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric --no-index --find-links=file:///kaggle/input/torch-geometric
#     !pip install torch_geometric_temporal  

Looking in links: file:///kaggle/input/torch-geometric
Processing /kaggle/input/torch-geometric/torch_scatter-2.1.1-cp310-cp310-linux_x86_64.whl
Processing /kaggle/input/torch-geometric/torch_sparse-0.6.17-cp310-cp310-linux_x86_64.whl
Processing /kaggle/input/torch-geometric/torch_cluster-1.6.1-cp310-cp310-linux_x86_64.whl
Processing /kaggle/input/torch-geometric/torch_spline_conv-1.2.2-cp310-cp310-linux_x86_64.whl
Processing /kaggle/input/torch-geometric/torch_geometric-2.3.1-py3-none-any.whl


# Import

In [4]:
import numpy as np
import pandas as pd
import os
import re
import json
from scipy.special import perm
from itertools import combinations,chain
from typing import List, Union
from torch_geometric.data import Data
from sklearn.preprocessing import OneHotEncoder

import matplotlib.pyplot as plt
import pickle
import time
from torch import nn
from torch.nn import Linear as Lin
from torch.nn import ReLU
from torch.nn import Sequential as Seq
import torch.nn.functional as F
from torch_geometric.nn import NNConv
from torch import autograd

from sklearn.model_selection import train_test_split

# DataLoader

In [5]:
Edge_Flag = List[Union[np.ndarray, None]]
Edge_Indices = List[Union[np.ndarray, None]]
Edge_Attr = List[Union[np.ndarray, None]]

Node_Flag = List[Union[np.ndarray, None]]
Node_Indices = List[Union[np.ndarray, None]]
Node_Attr = List[Union[np.ndarray, None]]
Additional_Attrs = List[np.ndarray]



class GraphSignal(object):
    # dynamic node static node attr dynamic edge and edge attr
    def __init__(
        self,
        edge_flag: Edge_Flag,
        edge_indices: Edge_Indices,
        edge_attr: Edge_Attr,
        node_flag: Node_Flag,
        node_indices: Node_Indices,
        node_attr: Node_Attr,
    ):
        
        self.edge_flag = edge_flag 
        self.edge_indices = torch.LongTensor(edge_indices).T.to(device)
        self.edge_attr = edge_attr
        self.node_flag = node_flag
        self.node_indices = torch.LongTensor(node_indices).to(device)
        self.node_attr = node_attr
        self.edge_attr_encoded = None
        self.node_attr_encoded = None
        
        self._set_snapshot_count()

    def _set_snapshot_count(self):
        self.snapshot_count = len(self.edge_flag)
    
    def encode_edge_attr(self, enocder):
        self.edge_attr_encoded = torch.FloatTensor(enocder.transform(self.edge_attr)).to(device)
    
    def encode_node_attr(self, enocder):
        self.node_attr_encoded = torch.FloatTensor(enocder.transform(self.node_attr)).to(device)
        
    def _get_edge_index(self, time_index: int):
        if time_index == 0:
            _start = 0
        else:
            _start = self.edge_flag[time_index-1]
        _end = self.edge_flag[time_index]
        _edge_index = self.edge_indices[:,_start:_end]
        return _edge_index

    # def _get_edge_weight(self, time_index: int):
    #     if self.edge_weights[time_index] is None:
    #         return self.edge_weights[time_index]
    #     else:
    #         return torch.FloatTensor(self.edge_weights[time_index])

    def _get_edge_attr(self, time_index: int):
        if self.edge_attr_encoded is None:
            print("Edge Attr Need to be Encoded!")
            raise
        if time_index == 0:
            _start = 0
        else:
            _start = self.edge_flag[time_index-1]
        _end = self.edge_flag[time_index]
        _edge_attr = self.edge_attr_encoded[_start:_end]
        return _edge_attr
    
    def _get_node_index_attr(self, time_index: int):
        if self.node_attr_encoded is None:
            print("Node Attr Need to be Encoded!")
            raise
        if time_index == 0:
            _start = 0
        else:
            _start = self.node_flag[time_index-1]
        _end = self.node_flag[time_index]
        _node_index = self.node_indices[_start:_end]
        _node_attr = self.node_attr_encoded[_node_index]
        return _node_index,_node_attr


    def __getitem__(self, time_index: int):
        edge_index = self._get_edge_index(time_index)
        edge_attr = self._get_edge_attr(time_index)
        node_index,node_attr = self._get_node_index_attr(time_index)

        snapshot = Data(
            edge_index=edge_index,
            edge_attr=edge_attr,
            node_index=node_index,
            node_attr=node_attr,
        )
        return snapshot

    def __next__(self):
        if self.t < self.snapshot_count:
            snapshot = self[self.t]
            self.t = self.t + 1
            return snapshot
        else:
            self.t = 0
            raise StopIteration

    def __iter__(self):
        self.t = 0
        return self
    
    def __len__(self):
        return self.snapshot_count


class GraphDatasetLoader(object):
    def __init__(self,input_path=""):
        self.input_path = input_path
        self._read_data()
    
    def _read_data(self):
        self._dataset = np.load(self.input_path)

    def get_dataset(self): # -> DynamicGraphTemporalSignal:
        dataset = GraphSignal(
            edge_flag = self._dataset['edge_flag'],
            edge_indices = self._dataset['edge_index'],
            edge_attr = self._dataset['edge_attr'],
            node_flag = self._dataset['node_flag'],
            node_indices = self._dataset['node_index'],
            node_attr = self._dataset['node_attr'],
        )
        return dataset

# Model

## MultiNNConv

In [6]:
class MultiNNConv(torch.nn.Module):
    def __init__(self, in_channels, out_channels, edge_channels, gcn_hidden_nums, edge_hidden_nums, lin_hidden_nums):
        super().__init__()
        
        def _create_edge_nn(edge_out_channels):
            edge_nn = Seq()
            pre_edge_h_num = edge_channels
            for edge_h_num in edge_hidden_nums:
                edge_nn.append(Lin(pre_edge_h_num,edge_h_num))
                edge_nn.append(ReLU())
                pre_edge_h_num = edge_h_num
            edge_nn.append(Lin(pre_edge_h_num,edge_out_channels))
            return edge_nn
        
        self.gcn_layers = nn.ModuleList()
        pre_h_num = in_channels
        for h_num in gcn_hidden_nums:
            edge_nn = _create_edge_nn(pre_h_num*h_num)
            self.gcn_layers.append(NNConv(pre_h_num, h_num, edge_nn, aggr='mean'))
            pre_h_num = h_num

        self.lin_net = Seq()
        for h_num in lin_hidden_nums[:-1]:
            self.lin_net.append(Lin(pre_h_num,h_num))
            pre_h_num = h_num
        self.lin_net.append(ReLU())
        self.lin_net.append(Lin(pre_h_num,out_channels))

    def forward(self, x, edge_index, edge_attr):
        out = x
        for conv in self.gcn_layers:
            out = conv(
                x=out,
                edge_index=edge_index,
                edge_attr=edge_attr,
            )
        out = self.lin_net(out)
        return out



class NNConvGRU(torch.nn.Module):
    def __init__(
        self,
        in_channels: int,
        out_channels: int,
        edge_channels: int,
        gcn_hidden_nums: List,
        edge_hidden_nums: List,
        lin_hidden_nums: List,
        normalization: str = "sym",
        bias: bool = True,
    ):
        super(NNConvGRU, self).__init__()

        self.in_channels = in_channels
        self.out_channels = out_channels
        self.edge_channels = edge_channels
        self.gcn_hidden_nums = gcn_hidden_nums
        self.edge_hidden_nums = edge_hidden_nums
        self.lin_hidden_nums = lin_hidden_nums
            
        self.normalization = normalization
        self.bias = bias
        self._create_parameters_and_layers()

    def _create_update_gate_parameters_and_layers(self):

        self.conv_x_z = MultiNNConv(
            in_channels = self.in_channels,
            out_channels = self.out_channels,
            edge_channels = self.edge_channels,
            gcn_hidden_nums = self.gcn_hidden_nums,
            edge_hidden_nums = self.edge_hidden_nums,
            lin_hidden_nums = self.lin_hidden_nums,
        )

        self.conv_h_z = MultiNNConv(
            in_channels = self.out_channels,
            out_channels = self.out_channels,
            edge_channels = self.edge_channels,
            gcn_hidden_nums = self.gcn_hidden_nums,
            edge_hidden_nums = self.edge_hidden_nums,
            lin_hidden_nums = self.lin_hidden_nums,
        )
        
    def _create_reset_gate_parameters_and_layers(self):

        self.conv_x_r = MultiNNConv(
            in_channels = self.in_channels,
            out_channels = self.out_channels,
            edge_channels = self.edge_channels,
            gcn_hidden_nums = self.gcn_hidden_nums,
            edge_hidden_nums = self.edge_hidden_nums,
            lin_hidden_nums = self.lin_hidden_nums,
        )

        self.conv_h_r = MultiNNConv(
            in_channels = self.out_channels,
            out_channels = self.out_channels,
            edge_channels = self.edge_channels,
            gcn_hidden_nums = self.gcn_hidden_nums,
            edge_hidden_nums = self.edge_hidden_nums,
            lin_hidden_nums = self.lin_hidden_nums,
        )

    def _create_candidate_state_parameters_and_layers(self):

        self.conv_x_h = MultiNNConv(
            in_channels = self.in_channels,
            out_channels = self.out_channels,
            edge_channels = self.edge_channels,
            gcn_hidden_nums = self.gcn_hidden_nums,
            edge_hidden_nums = self.edge_hidden_nums,
            lin_hidden_nums = self.lin_hidden_nums,
        )

        self.conv_h_h = MultiNNConv(
            in_channels = self.out_channels,
            out_channels = self.out_channels,
            edge_channels = self.edge_channels,
            gcn_hidden_nums = self.gcn_hidden_nums,
            edge_hidden_nums = self.edge_hidden_nums,
            lin_hidden_nums = self.lin_hidden_nums,
        )

    def _create_parameters_and_layers(self):
        self._create_update_gate_parameters_and_layers()
        self._create_reset_gate_parameters_and_layers()
        self._create_candidate_state_parameters_and_layers()

    def _set_hidden_state(self, X, H):
        if H is None:
            H = torch.zeros(X.shape[0], self.out_channels).to(X.device)
        return H

    def _calculate_update_gate(self, X, edge_index, edge_attr, H):
        Z = self.conv_x_z(X, edge_index, edge_attr)
        Z = Z + self.conv_h_z(H, edge_index, edge_attr)
        Z = torch.sigmoid(Z)
        return Z

    def _calculate_reset_gate(self, X, edge_index, edge_attr, H):
        R = self.conv_x_r(X, edge_index, edge_attr)
        R = R + self.conv_h_r(H, edge_index, edge_attr)
        R = torch.sigmoid(R)
        return R

    def _calculate_candidate_state(self, X, edge_index, edge_attr, H, R):
        H_tilde = self.conv_x_h(X, edge_index, edge_attr)
        H_tilde = H_tilde + self.conv_h_h(H * R, edge_index, edge_attr)
        H_tilde = torch.tanh(H_tilde)
        return H_tilde

    def _calculate_hidden_state(self, Z, H, H_tilde):
        H = Z * H + (1 - Z) * H_tilde
        return H

    def forward(
            self,
            X: torch.FloatTensor,
            edge_index: torch.LongTensor,
            edge_attr: torch.FloatTensor,
            H: torch.FloatTensor = None,
        ) -> torch.FloatTensor:
            H = self._set_hidden_state(X, H)
            Z = self._calculate_update_gate(X, edge_index, edge_attr, H)
            R = self._calculate_reset_gate(X, edge_index, edge_attr, H)
            H_tilde = self._calculate_candidate_state(X, edge_index, edge_attr, H, R)
            H = self._calculate_hidden_state(Z, H, H_tilde)
            return H

## TGAE

In [7]:
class TGAE(torch.nn.Module): # Not Heterogeneous
    def __init__(
        self, in_channels, out_channels, edge_channels, 
        embedding_hidden_nums, gnn_out_channels, deciding_hidden_nums,
        gru_gcn_hidden_nums, gru_edge_hidden_nums, gru_lin_hidden_nums):
        super(TGAE, self).__init__()
        
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.edge_channels = edge_channels
        self.embedding_hidden_nums = embedding_hidden_nums
        self.gnn_out_channels = gnn_out_channels
        self.deciding_hidden_nums = deciding_hidden_nums
        self.gru_gcn_hidden_nums = gru_gcn_hidden_nums
        self.gru_edge_hidden_nums = gru_edge_hidden_nums
        self.gru_lin_hidden_nums = gru_lin_hidden_nums
        
        # Encoder
        layers = []
        pre_h_num = in_channels
        for h_num in embedding_hidden_nums[:-1]:
#             layers.append(torch.nn.Dropout(p=0.2))
            layers.append(Lin(pre_h_num,h_num))
            layers.append(torch.nn.LeakyReLU())
            pre_h_num = h_num
        layers.append(Lin(pre_h_num,embedding_hidden_nums[-1]))
        self.encoder_embedding_net = Seq(*layers)
        
        self.encoder_gru = NNConvGRU(
            in_channels=embedding_hidden_nums[-1],
            out_channels=gnn_out_channels,
            edge_channels=edge_channels,
            gcn_hidden_nums=gru_gcn_hidden_nums,
            edge_hidden_nums=gru_edge_hidden_nums,
            lin_hidden_nums=gru_lin_hidden_nums,
        )

        layers = []
        pre_h_num = gnn_out_channels
        for h_num in deciding_hidden_nums:
#             layers.append(torch.nn.Dropout(p=0.2))
            layers.append(Lin(pre_h_num,h_num))
            layers.append(torch.nn.LeakyReLU())
            pre_h_num = h_num
        layers.append(Lin(pre_h_num,out_channels))
        self.encoder_deciding_net = Seq(*layers)
        
        # Decoder
        layers = []
        pre_h_num = out_channels
        for h_num in deciding_hidden_nums[::-1]:
            layers.append(torch.nn.Dropout(p=0.2))
            layers.append(Lin(pre_h_num,h_num))
            layers.append(torch.nn.LeakyReLU())
            pre_h_num = h_num
        layers.append(Lin(pre_h_num,gnn_out_channels))
        self.decoder_deciding_net = Seq(*layers)
        
        self.decoder_gru = NNConvGRU(
            in_channels=gnn_out_channels,
            out_channels=embedding_hidden_nums[-1],
            edge_channels=edge_channels,
            gcn_hidden_nums=gru_gcn_hidden_nums,
            edge_hidden_nums=gru_edge_hidden_nums,
            lin_hidden_nums=gru_lin_hidden_nums,
        )
        
        layers = []
        pre_h_num = embedding_hidden_nums[-1]
        for h_num in embedding_hidden_nums[:-1][::-1]:
            layers.append(torch.nn.Dropout(p=0.2))
            layers.append(Lin(pre_h_num,h_num))
            layers.append(torch.nn.LeakyReLU())
            pre_h_num = h_num
        layers.append(Lin(pre_h_num,in_channels))
        self.decoder_embedding_net = Seq(*layers)
        

    def forward(self, x, edge_index, edge_attr, h_encoder=None, h_decoder=None):
        """
        x = Node features for T time steps
        edge_index = Graph edge indices
        """
        # Encoder
        out = self.encoder_embedding_net(x)
        
        # GNN layer
        h_encoder = self.encoder_gru(out, edge_index, edge_attr, h_encoder) 
        
        out = self.encoder_deciding_net(h_encoder)
        
        out = self.decoder_deciding_net(out)
        
        # TODO Reverse Edge Index 
        
        h_decoder = self.decoder_gru(out, torch.flip(edge_index,dims=(0,)), edge_attr, h_decoder)
        
        out = self.decoder_embedding_net(h_decoder)

        return out, h_encoder, h_decoder

# Training

## Global State Function

In [25]:
def create_hidden_global(num_node, out_channels):
    hidden_global = torch.FloatTensor(np.zeros([num_node,out_channels])).to(device)
#     hidden_global =torch.zeros([num_node,out_channels], dtype=torch.float).to(device)
    return hidden_global

def select_hidden_local(hidden_global, index):
#     h = hidden_global[index] #REGULAR INDEXING
    h = hidden_global.index_select(dim=0, index=index) #INDEX SELECT
    return h

# TODO: Aggregation of hidden and cell
def update_hidden_gobal(hidden_global, h, index):
    # hidden_global[index] = h.detach() #REGULAR INDEXING
    # for key,value in mapping.items():
    #     hidden_global[value] = h[key] 
    hidden_global.index_copy_(dim=0, index=index, source=h)

## Read Data

In [18]:
data_dir_0 = '/kaggle/input/dissertation-data'
data_dir_1 = '2021-09-11-umbrella-experiment-32run-fran'


signals = []
y = []
with open(os.path.join(data_dir_0, data_dir_1, "annotated.json")) as f:
    annotated_dict = json.load(f)

for data_dir_2 in os.listdir(os.path.join(data_dir_0, data_dir_1)):
    if data_dir_2 == "annotated.json":
        continue
    r = re.compile(".*.npz")
    graph_files = list(filter(r.match, os.listdir(os.path.join(data_dir_0, data_dir_1, data_dir_2))))

    if len(graph_files) > 1:
        print("Multiple Graph Files!")
        raise
    if len(graph_files) == 0:
        print("Not Found Graph File!")
        raise

    dataloader = GraphDatasetLoader(os.path.join(data_dir_0, data_dir_1, data_dir_2, graph_files[0]))
    signal = dataloader.get_dataset()
    signals.append(signal)
    y.append(annotated_dict[data_dir_2])

# split train and test dataset
signals_train, signals_test, y_train, y_test = train_test_split(signals, y, test_size=0.2, random_state=1)
signals_train, signals_val, y_train, y_val = train_test_split(signals_train, y_train, test_size=0.25, random_state=1) # 0.25 x 0.8 = 0.2

In [19]:
node_attr_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
node_attr_encoder = node_attr_encoder.fit(np.concatenate([sample.node_attr for sample in signals_train]))

edge_attr_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
edge_attr_encoder = edge_attr_encoder.fit(np.concatenate([sample.edge_attr for sample in signals_train]))

for signal in signals_train:
    signal.encode_node_attr(node_attr_encoder)
    signal.encode_edge_attr(edge_attr_encoder)

X_train = []
for signal in signals_train:
    X_train.append(list(signal))

nums_node_train = []
for signal in signals_train:
    nums_node_train.append(signal.node_attr.shape[0])

In [20]:
# for sample in X_train[:1]:
#     for snapshot in sample:
#         node_attr = snapshot.node_attr
#         node_index = snapshot.node_index
#         edge_attr = snapshot.edge_attr
#         edge_index = snapshot.edge_index
        
#         print(f"node_attr: {node_attr.shape}")
#         print(f"node_index: {node_index.shape}")
#         print(f"edge_attr: {edge_attr.shape}")
#         print(f"edge_index: {edge_index.shape}")

In [21]:
NUM_EPOCH = 10
IN_CHANNELS = signals_train[0].node_attr_encoded.shape[1]
EDGE_CHANNELS = signals_train[0].edge_attr_encoded.shape[1]

In [22]:
model = TGAE(
    in_channels=IN_CHANNELS, 
    out_channels=5, 
    edge_channels=EDGE_CHANNELS, 
    embedding_hidden_nums=[4,4],
    gnn_out_channels=8,
    deciding_hidden_nums=[4,4],
    gru_gcn_hidden_nums=[16,16],
    gru_edge_hidden_nums=[32],
    gru_lin_hidden_nums=[64,64],
)

loss_f = torch.nn.MSELoss(reduction = 'none')
optimizer = torch.optim.Adam(model.parameters(), lr = 2e-4, weight_decay=1e-5)

model = model.to(device)
loss_f = loss_f.to(device)
# print(model)

In [33]:
# train_loop
def train_loop(sample, num_node, state_channels, model, loss_fn, optimizer, device):

    hidden_encoder_global = create_hidden_global(num_node=num_node,out_channels=state_channels)
    hidden_decoder_global = create_hidden_global(num_node=num_node,out_channels=model.embedding_hidden_nums[-1])
    # cell_global = create_cell_global(num_nodes=len(global_nodes),out_channels=256)
    
    train_losses = []
    total_loss = 0
    model.train()
    for i,snapshot in enumerate(sample,start=1):
        if snapshot.node_attr.shape[0] == 0:
            # print("snapshot_{} has no data...".format(i))
            continue

        _node_index = snapshot.node_index        
        _node_attr = snapshot.node_attr
        _edge_attr = snapshot.edge_attr
        _edge_index =  snapshot.edge_index
        
        pre_h_encoder = select_hidden_local(hidden_encoder_global, _node_index)
        pre_h_decoder = select_hidden_local(hidden_decoder_global, _node_index)
        
        # print(_node_attr.type())
        # Compute prediction and loss
        outs, h_encoder, h_decoder = model(_node_attr,_edge_index,_edge_attr)

        update_hidden_gobal(hidden_encoder_global, h_encoder.detach(), _node_index)
        update_hidden_gobal(hidden_decoder_global, h_decoder.detach(), _node_index)

#         train_loss = 0
#         for i in range(_node_attr.shape[0]):
# #             train_loss += torch.sqrt(loss_f(torch.log(_node_attr[i]+1), torch.log(outs[i]+1)))
#             train_loss += torch.sqrt(torch.sum(loss_f(_node_attr[i], outs[i])))
        train_loss = torch.sum(torch.sqrt(torch.sum(loss_f(_node_attr, outs),dim=1)))
        total_loss += train_loss
        train_losses.append(train_loss.cpu().detach().numpy())
    
        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()
    return train_losses

In [ ]:
for i_epoch in range(1,NUM_EPOCH+1):
    train_losses = []
    _start = time.time()
    for sample,y, num_node in zip(X_train,y_train,nums_node_train):
        train_loss = train_loop(sample, num_node, model.gnn_out_channels, model, loss_f, optimizer, device)
#         print(np.mean(train_loss))
        train_losses += train_loss
    _end = time.time()
    print(f"{i_epoch}/{NUM_EPOCH}: train loss {np.mean(train_losses):.4f} cost {_end-_start:.4f}s")

1/10: train loss 95.2464 cost 45.4459s
2/10: train loss 77.9003 cost 44.2892s
3/10: train loss 58.1484 cost 44.4208s
4/10: train loss 56.4516 cost 44.2751s
5/10: train loss 54.7992 cost 44.3660s
6/10: train loss 50.0876 cost 44.5210s
7/10: train loss 32.8199 cost 44.3605s
8/10: train loss 29.3768 cost 44.7493s


In [ ]:
def test_loop(test_set, num_nodes, state_channels, model, loss_fn, device):
    size = len(train_set)

    hidden_encoder_global = create_hidden_global(num_nodes=num_nodes,out_channels=state_channels)
    hidden_decoder_global = create_hidden_global(num_nodes=num_nodes,out_channels=state_channels)
    # cell_global = create_cell_global(num_nodes=len(global_nodes),out_channels=256)
    
    test_losses = []
    with torch.no_grad():
        for i,snapshot in enumerate(test_set,start=1):
            if snapshot.node_attr.shape[0] == 0:
                # print("snapshot_{} has no data...".format(i))
                continue

            node_attr = snapshot.node_attr
            node_index = snapshot.node_index
            edge_attr = snapshot.edge_attr
            edge_index = snapshot.edge_index


            _node_attr = node_attr.to(device)
            _edge_attr = edge_attr.to(device)
            _edge_index = edge_index.to(device)

            pre_h_encoder = torch.tensor(select_hidden_local(hidden_encoder_global, node_index),dtype=torch.float32).to(device)
            pre_h_decoder = torch.tensor(select_hidden_local(hidden_decoder_global, node_index),dtype=torch.float32).to(device)

            # Compute prediction and loss
            outs, h_encoder, h_decoder = model(_node_attr,_edge_index,_edge_attr)

            update_hidden_gobal(hidden_encoder_global, h_encoder, node_index)
            update_hidden_gobal(hidden_decoder_global, h_decoder, node_index)

            test_loss = 0
            for i in range(node_attr.shape[0]):
    #             train_loss += torch.sqrt(loss_f(torch.log(_node_attr[i]+1), torch.log(outs[i]+1)))
                test_loss += torch.sqrt(loss_f(_node_attr[i], outs[i]))
        
        if(use_gpu):
            test_losses.append(test_loss.cpu().detach().numpy())
        else:
            test_losses.append(test_loss.detach().numpy())
    return test_losses
        